# AI Technical Assessment

## Load and preprocess the pdf document

### Subtask:
Load the specified PDF document, preprocess it to clean and prepare the text for chunking.


**Reasoning**:
The first step is to load the PDF and extract the text content from each page. I will use PyMuPDF for this purpose.



In [ ]:
import fitz # PyMuPDF
import re

pdf_path = "/content/HSC26 Bangla 1st Paper.pdf"
doc = fitz.open(pdf_path)

text_content = ""
for page_num in range(doc.page_count):
    page = doc.load_page(page_num)
    text_content += page.get_text()

# Basic cleaning: remove extra whitespace and newlines
cleaned_text = re.sub(r'\s+', ' ', text_content).strip()

print(f"Original text length: {len(text_content)}")
print(f"Cleaned text length: {len(cleaned_text)}")
print("First 500 characters of cleaned text:")
print(cleaned_text[:500])

Original text length: 82287
Cleaned text length: 79499
First 500 characters of cleaned text:
অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জিজ্ঞাাসা , অপরিরিতা আল ািয রিষয় িাাং া ১ম পত্র ১। অনুপলেি িািা কী কলি জীরিকা রনিবাহ কিলতন? ক) ডাক্তার্ি খ) ওকালর্ত গ) মাস্টার্ি ঘ) ব্যব্সা ২। োোলক ভাগ্য দেিতাি প্রধান এলজন্ট ি াি কািণ, তাি- ক) প্রর্তপজি খ) প্রভাব্ গ) র্ব্চক্ষণতা ঘ) কূট ব্ুর্ি র্নকচি অনুকেদটি পক়ে ৩ ও ৪ সংখযক প্রকেি উিি দাও। র্পতৃহীন দীপুি চাচাই র্িকলন পর্িব্াকিি কতিা। দীপু র্িজক্ষত হকলও তাি র্সিান্ত যনও াি ক্ষমতা র্িল না। চাচা তাি র্ব্ক ি উকদযাগ র্নকলও যেৌতুক র্নক ব্া়োব্ার়্ে কিাি কািকণ কনযাি


## Chunk the document

### Subtask:
Split the preprocessed document into smaller, manageable chunks.


**Reasoning**:
Determine chunk size and overlap, then split the cleaned text into chunks and store them in a list. Finally, print the number of chunks and the first chunk.



In [ ]:
# Determine chunk size and overlap
chunk_size = 500  # characters
chunk_overlap = 100 # characters

# Split the cleaned text into chunks
chunks = []
for i in range(0, len(cleaned_text), chunk_size - chunk_overlap):
    chunk = cleaned_text[i:i + chunk_size]
    chunks.append(chunk)

# Print the number of chunks and the first chunk
print(f"Number of chunks created: {len(chunks)}")
if chunks:
    print("\nFirst chunk:")
    print(chunks[0])

Number of chunks created: 199

First chunk:
অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জিজ্ঞাাসা , অপরিরিতা আল ািয রিষয় িাাং া ১ম পত্র ১। অনুপলেি িািা কী কলি জীরিকা রনিবাহ কিলতন? ক) ডাক্তার্ি খ) ওকালর্ত গ) মাস্টার্ি ঘ) ব্যব্সা ২। োোলক ভাগ্য দেিতাি প্রধান এলজন্ট ি াি কািণ, তাি- ক) প্রর্তপজি খ) প্রভাব্ গ) র্ব্চক্ষণতা ঘ) কূট ব্ুর্ি র্নকচি অনুকেদটি পক়ে ৩ ও ৪ সংখযক প্রকেি উিি দাও। র্পতৃহীন দীপুি চাচাই র্িকলন পর্িব্াকিি কতিা। দীপু র্িজক্ষত হকলও তাি র্সিান্ত যনও াি ক্ষমতা র্িল না। চাচা তাি র্ব্ক ি উকদযাগ র্নকলও যেৌতুক র্নক ব্া়োব্ার়্ে কিাি কািকণ কনযাি


## Vectorize the chunks

### Subtask:
Generate vector representations (embeddings) for each document chunk.


**Reasoning**:
Generate vector representations (embeddings) for each document chunk using a multilingual sentence transformer model.



In [ ]:
from sentence_transformers import SentenceTransformer

# Load a suitable multilingual model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Generate embeddings for each chunk
embeddings = []
for chunk in chunks:
    embedding = model.encode(chunk)
    embeddings.append(embedding)

# Print the number of embeddings and the shape of the first embedding
print(f"Number of embeddings generated: {len(embeddings)}")
if embeddings:
    print(f"Shape of the first embedding: {embeddings[0].shape}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Number of embeddings generated: 199
Shape of the first embedding: (384,)


# Create a Vector Database

In [ ]:
from langchain_community.vectorstores import FAISS
import numpy as np

# Convert embeddings to numpy array
embeddings_np = np.array(embeddings)

# Create FAISS index
vectorstore = FAISS.from_embeddings(text_embeddings=list(zip(chunks, embeddings_np)), embedding=model)


# Verify the creation of the vector database
print(f"Vector database created with {vectorstore.index.ntotal} vectors.")

Vector database created with 199 vectors.


## Implement the rag system

### Subtask:
Implement the rag system


**Reasoning**:
Define the RAG function, generate query embedding, perform similarity search, construct the prompt, use a language model (assuming a simple one for demonstration), and return the response.



In [ ]:
from transformers import pipeline

# Load a simple conversational model (adjust as needed)
# Note: A more sophisticated model or API would be needed for better performance
# This is a placeholder for demonstration purposes.
# Using a text generation pipeline
# You might need to install a model like 'google/flan-t5-small' or similar if you don't have one
# !pip install transformers accelerate
# !pip install bitsandbytes # if using 8-bit quantization

# Attempt to load a model. If this fails, you may need to specify a different model or handle authentication/installation.
try:
    generator = pipeline("text-generation", model="google/flan-t5-small", device=0) # Use device=0 for GPU if available
except Exception as e:
    print(f"Could not load text-generation pipeline: {e}")
    print("Falling back to a simpler approach or requiring manual model setup.")
    generator = None # Set generator to None if loading fails


def rag_query(query: str, k: int = 3) -> str:
    """
    Processes a user query using the RAG system.

    Args:
        query: The user's query string (English or Bengali).
        k: The number of relevant document chunks to retrieve.

    Returns:
        The generated response from the language model.
    """
    if model is None or vectorstore is None or generator is None:
        return "RAG system not fully initialized. Model, vector store, or generator is missing."

    # 2. Generate embedding for the user query
    query_embedding = model.encode(query)

    # 3. Perform similarity search
    # The vectorstore object from FAISS.from_embeddings does not have 'similarity_search'.
    # We need to use the underlying FAISS index or query the retriever interface if one was set up.
    # Assuming vectorstore is a LangChain FAISS object, it should have a similarity_search method.
    # Let's verify this based on the previous step's output indicating `FAISS.from_embeddings` was used.
    try:
        retrieved_docs = vectorstore.similarity_search_by_vector(query_embedding, k=k)
    except AttributeError:
         # If similarity_search_by_vector is not available, we might need a different approach
         # or there was an issue with the FAISS object creation.
         # For demonstration, let's assume the expected method exists based on LangChain FAISS docs.
         # If this fails in execution, the FAISS object from the previous step needs re-evaluation.
         print("Error: 'similarity_search_by_vector' method not found on the vectorstore object.")
         return "Error retrieving documents."


    # 4. Construct a prompt
    context = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

    # Simple prompt template - instruct the model to use context and the query language
    prompt_template = f"""Use the following context to answer the user's query.
    Respond in the same language as the query.
    If you don't know the answer based on the context, just say that you don't know.

    Context:
    {context}

    Query:
    {query}

    Answer:
    """

    # 5. Use a language model to generate a response
    if generator:
        # Using the text-generation pipeline
        # Adjust max_new_tokens and other parameters as needed
        response = generator(prompt_template, max_new_tokens=150, num_return_sequences=1)[0]['generated_text']
        # The generated text might include the prompt itself, need to clean it.
        # A simple cleaning might be to remove the prompt_template part.
        # However, the pipeline often just continues the text.
        # A better approach might be to fine-tune the prompt or use a different pipeline/model structure.
        # For this simple demo, let's just assume the model generates after "Answer:".
        # Finding the "Answer:" and taking the text after it.
        answer_prefix = "Answer:"
        if answer_prefix in response:
            response = response.split(answer_prefix, 1)[1].strip()

    else:
        response = "Language model generator not initialized."

    # 6. Return the generated response
    return response

# Example usage (optional, for testing)
# english_query = "What is the name of the chapter about humans?"
# bangla_query = "ঐক্যতান কবিতার মূলভাব কী?"
#
# print(f"English Query: {english_query}")
# english_response = rag_query(english_query)
# print(f"Response: {english_response}")
#
# print(f"\nBengali Query: {bangla_query}")
# bangla_response = rag_query(bangla_query)
# print(f"Response: {bangla_response}")


Device set to use cpu
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'Gemma3nForConditionalGeneration', 'Gemma3nForCausa

In [ ]:
from transformers import pipeline

# Load a simple conversational model suitable for Flan-T5
# Using 'text2text-generation' pipeline for Flan-T5
try:
    generator = pipeline("text2text-generation", model="google/flan-t5-small", device=0) # Use device=0 for GPU if available
    print("Text2Text generation pipeline loaded successfully.")
except Exception as e:
    print(f"Could not load text2text-generation pipeline: {e}")
    print("Language model generator not initialized.")
    generator = None # Set generator to None if loading fails


def rag_query(query: str, k: int = 3) -> str:
    """
    Processes a user query using the RAG system.

    Args:
        query: The user's query string (English or Bengali).
        k: The number of relevant document chunks to retrieve.

    Returns:
        The generated response from the language model.
    """
    # Check if all necessary components are initialized
    if 'model' not in globals() or model is None:
         return "SentenceTransformer model not initialized."
    if 'vectorstore' not in globals() or vectorstore is None:
        return "Vector store not initialized."
    if generator is None:
        return "Language model generator not initialized."


    # 2. Generate embedding for the user query
    query_embedding = model.encode(query)

    # 3. Perform similarity search
    try:
        # Assuming vectorstore is a LangChain FAISS object which has this method
        retrieved_docs = vectorstore.similarity_search_by_vector(query_embedding, k=k)
    except AttributeError:
         print("Error: 'similarity_search_by_vector' method not found on the vectorstore object.")
         return "Error retrieving documents."
    except Exception as e:
         print(f"An error occurred during similarity search: {e}")
         return "Error retrieving documents."


    # 4. Construct a prompt
    context = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

    # Simple prompt template - instruct the model to use context and the query language
    # For Flan-T5, a question answering format might be more effective.
    prompt_template = f"""Answer the following question based on the provided context.
    Respond in the same language as the question.
    If the answer is not in the context, say "I cannot answer this question based on the provided information."

    Context:
    {context}

    Question:
    {query}

    Answer:
    """

    # 5. Use a language model to generate a response
    if generator:
        try:
            # Using the text2text-generation pipeline
            # Adjust max_new_tokens and other parameters as needed
            # Flan-T5 is an encoder-decoder model, so the output is the generated text directly.
            response = generator(prompt_template, max_new_tokens=150, num_return_sequences=1)[0]['generated_text']
        except Exception as e:
            print(f"An error occurred during text generation: {e}")
            response = "Error generating response."
    else:
        response = "Language model generator not initialized." # This case should be caught earlier, but kept for safety.

    # 6. Return the generated response
    return response

# Example usage (optional, for testing)
# english_query = "What is the name of the chapter about humans?"
# bangla_query = "ঐক্যতান কবিতার মূলভাব কী?"
#
# print(f"English Query: {english_query}")
# english_response = rag_query(english_query)
# print(f"Response: {english_response}")
#
# print(f"\nBengali Query: {bangla_query}")
# bangla_response = rag_query(bangla_query)
# print(f"Response: {bangla_response}")

Device set to use cpu


Text2Text generation pipeline loaded successfully.


## (bonus) implement a conversation api

### Subtask:
Create a simple REST API to interact with the RAG system.


**Reasoning**:
Import necessary libraries and define the FastAPI application instance and the request body model.



In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

app = FastAPI()

class QueryRequest(BaseModel):
    query: str

**Reasoning**:
Define the API endpoint that accepts the user's query, calls the rag_query function, and returns the response.



In [ ]:
@app.post("/query")
def process_query(request: QueryRequest):
    """
    API endpoint to process user queries using the RAG system.
    """
    user_query = request.query
    response = rag_query(user_query)
    return {"response": response}

# Include the code to run the FastAPI application
if __name__ == "__main__":
    # Note: In a production environment, you might want to configure host and port
    # and potentially use a process manager like Gunicorn.
    # For this example, we'll run directly with uvicorn.
    uvicorn.run(app, host="0.0.0.0", port=3000)

INFO:     Started server process [14877]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:3000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [14877]


**Reasoning**:
The error `RuntimeError: asyncio.run() cannot be called from a running event loop` indicates that `uvicorn.run()` is being called within an environment that already has a running asyncio event loop, such as a Jupyter Notebook. To run the FastAPI application within this environment for testing or demonstration purposes without blocking the notebook execution, we can use `nest_asyncio`.



In [ ]:
import nest_asyncio
nest_asyncio.apply()

# This part is only for running in environments like Jupyter notebooks
# In a typical application, you would just use the if __name__ == "__main__": block
# from the previous step in a separate Python file.
print("FastAPI app defined. To run it, execute the cell below.")

# To run the app in a notebook, you can use a different approach if nest_asyncio is not preferred
# or if you want to run it in a separate thread/process.
# However, for simplicity and demonstration within the notebook context, nest_asyncio is used here.

FastAPI app defined. To run it, execute the cell below.


## (bonus) implement rag evaluation

### Subtask:
Develop a method to evaluate the groundedness and relevance of the generated answers.


**Reasoning**:
Define a function `evaluate_rag_answer` that takes query, context, and answer as input and uses keyword overlap and semantic similarity to assess groundedness and relevance.



In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def evaluate_rag_answer(query: str, retrieved_context: str, generated_answer: str, model) -> dict:
    """
    Evaluates the groundedness and relevance of a generated answer.

    Args:
        query: The original user query.
        retrieved_context: The text retrieved from the document.
        generated_answer: The answer generated by the language model.
        model: The sentence transformer model for generating embeddings.

    Returns:
        A dictionary containing evaluation metrics (groundedness and relevance scores).
    """
    eval_results = {
        "groundedness_score": 0.0,
        "relevance_score": 0.0,
        "groundedness_judgment": "Unsupported",
        "relevance_judgment": "Irrelevant"
    }

    if not generated_answer or not retrieved_context:
        # Cannot evaluate if there's no answer or context
        return eval_results

    # --- Groundedness Evaluation ---
    # Method 1: Keyword Overlap (Simple approach)
    # Check if key terms from the answer are present in the context
    answer_words = set(re.findall(r'\b\w+\b', generated_answer.lower()))
    context_words = set(re.findall(r'\b\w+\b', retrieved_context.lower()))
    common_words = answer_words.intersection(context_words)
    # Simple groundedness score based on word overlap ratio (can be refined)
    if len(answer_words) > 0:
        eval_results["groundedness_score"] = len(common_words) / len(answer_words)

    # Method 2: Semantic Similarity (Using sentence embeddings)
    # Compare the similarity between the answer and the context
    try:
        answer_embedding = model.encode(generated_answer)
        context_embedding = model.encode(retrieved_context)
        # Reshape for cosine_similarity calculation if they are 1D arrays
        if answer_embedding.ndim == 1:
             answer_embedding = answer_embedding.reshape(1, -1)
        if context_embedding.ndim == 1:
             context_embedding = context_embedding.reshape(1, -1)

        # Calculate cosine similarity
        semantic_groundedness = cosine_similarity(answer_embedding, context_embedding)[0][0]
        # Combine semantic similarity with keyword overlap (optional, adjust weighting)
        # eval_results["groundedness_score"] = (eval_results["groundedness_score"] + semantic_groundedness) / 2
        eval_results["semantic_groundedness_score"] = float(semantic_groundedness)

        # Judgment based on a threshold (can be refined)
        if eval_results["groundedness_score"] > 0.2 or semantic_groundedness > 0.5: # Example thresholds
             eval_results["groundedness_judgment"] = "Supported"
        else:
             eval_results["groundedness_judgment"] = "Unsupported"

    except Exception as e:
        print(f"Error during groundedness semantic similarity calculation: {e}")


    # --- Relevance Evaluation ---
    # Method 1: Keyword Overlap (Simple approach)
    # Check if key terms from the query are present in the answer
    query_words = set(re.findall(r'\b\w+\b', query.lower()))
    answer_words_for_relevance = set(re.findall(r'\b\w+\b', generated_answer.lower())) # Recalculate to be safe
    common_query_answer_words = query_words.intersection(answer_words_for_relevance)
    # Simple relevance score based on word overlap ratio (can be refined)
    if len(query_words) > 0:
        eval_results["relevance_score"] = len(common_query_answer_words) / len(query_words)


    # Method 2: Semantic Similarity (Using sentence embeddings)
    # Compare the similarity between the query and the answer
    try:
        query_embedding = model.encode(query)
        # Ensure query_embedding is 2D for cosine_similarity
        if query_embedding.ndim == 1:
            query_embedding = query_embedding.reshape(1, -1)

        semantic_relevance = cosine_similarity(query_embedding, answer_embedding)[0][0]
        # Combine semantic similarity with keyword overlap (optional)
        # eval_results["relevance_score"] = (eval_results["relevance_score"] + semantic_relevance) / 2
        eval_results["semantic_relevance_score"] = float(semantic_relevance)

        # Judgment based on a threshold (can be refined)
        if eval_results["relevance_score"] > 0.2 or semantic_relevance > 0.5: # Example thresholds
            eval_results["relevance_judgment"] = "Relevant"
        else:
            eval_results["relevance_judgment"] = "Irrelevant"

    except Exception as e:
        print(f"Error during relevance semantic similarity calculation: {e}")


    return eval_results

# Example Usage (for testing)
# Note: Replace with actual model and sample data
# try:
#     # Assuming 'model' is already loaded from previous steps
#     sample_query_en = "What is the main theme of the chapter?"
#     sample_context_en = "The main theme of this chapter is the importance of education..."
#     sample_answer_en = "The main theme discussed is the significance of learning."
#
#     sample_query_bn = "কবিতার মূলভাব কী?"
#     sample_context_bn = "কবিতাটির মূলভাব হলো প্রকৃতির রূপ বর্ণনা..."
#     sample_answer_bn = "এই কবিতার মূলভাব হলো প্রকৃতির সৌন্দর্য।"
#
#     print("\n--- English Example ---")
#     eval_en = evaluate_rag_answer(sample_query_en, sample_context_en, sample_answer_en, model)
#     print(f"Evaluation Results (English): {eval_en}")
#
#     print("\n--- Bengali Example ---")
#     eval_bn = evaluate_rag_answer(sample_query_bn, sample_context_bn, sample_answer_bn, model)
#     print(f"Evaluation Results (Bengali): {eval_bn}")
#
# except NameError:
#     print("\nSentenceTransformer model 'model' is not defined. Please run the vectorization step first.")
# except Exception as e:
#      print(f"\nAn error occurred during example evaluation: {e}")


In [ ]:
# Install ngrok
!pip install pyngrok

In [ ]:
# Authenticate ngrok (You'll need an ngrok auth token from your ngrok account)
# Replace 'YOUR_AUTHTOKEN' with your actual ngrok auth token
from pyngrok import ngrok
from google.colab import userdata

# Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
# Add your token to Colab secrets with the name 'NGROK_AUTH_TOKEN'
NGROK_AUTH_TOKEN = userdata.get("NGROK_AUTH_TOKEN")
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start ngrok tunnel for port 8000
# The ngrok process will run in the background
ngrok_tunnel = ngrok.connect(3000)
print(f"Public URL: {ngrok_tunnel.public_url}")

Public URL: https://9f8d4f544c66.ngrok-free.app


In [6]:
!git clone https://github.com/Mostafa-Annur/AI_Assessment.git

Cloning into 'AI_Assessment'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 8 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (8/8), 12.84 KiB | 4.28 MiB/s, done.


In [8]:
# Install git (if not already installed, Colab usually has it)
# !apt-get update && apt-get install -y git

# Configure Git (replace with your name and email)
!git config --global user.name "Mostafa-Annur"
!git config --global user.email "mostafaannur@gmail.com"

# Clone your empty GitHub repository (replace with your repo URL)
# You might need to use a Personal Access Token for authentication if prompted
!git clone https://github.com/Mostafa-Annur/AI_Assessment.git

# Navigate into your repository directory
import os
os.chdir('AI_Assessment')

# Copy your notebook file into the repository directory
# Replace 'your_notebook_name.ipynb' with the actual filename of your notebook
!cp /content/AI_Assessment.ipynb .

# Add the notebook file
# Replace 'your_notebook_name.ipynb' with the actual filename of your notebook
!git add your_notebook_name.ipynb

# Commit the changes
!git commit -m "Add RAG notebook"

# Push to the remote repository
!git push origin main # or master, depending on your branch name

# Note: Uncomment and execute the commands above one by one after replacing placeholders.

Cloning into 'AI_Assessment'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 8 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (8/8), 12.84 KiB | 12.84 MiB/s, done.
cp: cannot stat '/content/AI_Assessment.ipynb': No such file or directory
fatal: pathspec 'your_notebook_name.ipynb' did not match any files
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address
